### Data Wrangling : Join , Combine And Reshape

Hierarchical indexing is like having a super-organized wardrobe. It’s great for:

- **Reshaping data**: Using `stack()` and `unstack()` to switch between Series and DataFrame formats.
- **Group-based operations**: Like creating pivot tables (more on this in Section 8.3).
- **Handling complex datasets**: When data has multiple categories (e.g., sales by region and product).
---
- **MultiIndex is like a family tree**:  
  Outer level (`a`, `b`, `c`) is like parents, inner level (`1`, `2`, `3`) is like kids.
- **Unstack = spread out**:  
  Like laying out all your rotis on a table.
- **Stack = fold back**:  
  Like stacking the rotis back into a pile.
- **Partial indexing = selective munching**:  
  Pick only the gulab jamuns you want from the sweet box!
- **Names matter**:  
  Giving names to index levels (`key1`, `key2`) is like labeling your dabbas so you don’t mix up dal with sabzi.

In [ ]:
import pandas as pd
import numpy as np
data = pd.Series(np.random.uniform(size=9),index=[["a", "a", "a", "b", "b", "c", "c", "d","d"],[1, 2, 3, 1, 3, 1, 2, 2, 3]])
data.index #MultiIndex.from_tuples([("a", 1), ("a", 2), ("a", 3), ("b", 1), ("b", 3), ("c", 1), ("c", 2), ("d", 2), ("d", 3)])
data['b'] #Indexing
data['b':'d'] #Slicing
data[['a','d']] #Fancy indexing
data.loc[:,2] #Partial indexing
data.loc['a':'b', 1:2] #Partial slicing
data.unstack() #Unstacking - converts to DataFrame
(data.unstack()).stack() #Stacking - converts to Series

frame = pd.DataFrame(np.arange(12).reshape((4, 3)),index=[["a", "a", "b", "b"], [1, 2, 1, 2]],columns=[["Ohio", "Ohio", "Colorado"],["Green", "Red", "Green"]])
frame.index.names = ['key1','key2'] #Setting names for index levels
frame.columns.names = ['state','color'] #Setting names for column levels
pd.MultiIndex.from_arrays([['Ohio' ,  'Ohio' , 'Colorado'],['Green','Red','Green']],names = ['States','Color'])

**Desi Tips to Remember**
- **swaplevel** = seat exchange: It’s just flipping two index levels, like swapping roti and naan plates. Data stays the same!
- **sort_index** = organizing your dabba: Sort by one level or all levels to keep things neat and tidy.
- **Sorting by level=0** = speed booster: It’s like keeping your masala box sorted so you grab spices faster.
- **groupby with level** = summing by category: Like adding up all mithai by type (gulab jamun, jalebi) or meal time (lunch, dinner).
- **Axis matters**: Use axis="columns" for column-level operations, else it’s rows by default.

In [ ]:
frame.swaplevel('key1','key2') #Using lables names
frame.swaplevel(0,1) #Using integer positions
frame.sort_index(level= 1) #Sorting by level 1
frame.swaplevel(0,1).sort_index(level=0) #Sorting by level 0 after swapping levels
frame.groupby(level=0).sum() #Grouping by level 0
frame.groupby(level = 'color',axis = 'columns').sum() #This is not allowed now you have to use transpose

**Desi Breakdown – Yeh Sab Kyun Useful Hai?**
- **set_index**: Use this when a column (or two) makes more sense as the index.
- **set_index**= turn columns indexes to row indexes and vice versa
- **drop=False** -> to keep the columns as columns too (not removing them)
- **reset_index** = flattning back to dataframe(2D)
- **Use case** = organize or flatten: set_index organizes data hierarchically; reset_index makes it simple and flat.

In [ ]:
frame = pd.DataFrame({
    "a": range(7),
    "b": range(7, 0, -1),
    "c": ["one", "one", "one", "two", "two", "two", "two"],
    "d": [0, 1, 2, 0, 1, 2, 3]
})
frame2 = frame.set_index(['c', 'd'])
frame2 = frame.set_index(['c','d'],drop = False) #Setting index without dropping the columns
frame2.reset_index() #Resetting index -> flatten to dataframe(you can use on multiindex too)

**Merge Arguments Cheat Sheet**
- **left**: Left DataFrame.
- **right**: Right DataFrame.
- **how**: Join type (inner, left, right, outer).
- **on**: Column(s) to join on (if same in both).
- **left_on/right_on**: Columns to join on if names differ.
- **suffixes**: Strings to add to overlapping column names (default: _x, _y).
- **sort**: Sort result by join keys (default: False).
- **validate**: Check if merge is one-to-one, one-to-many, etc.
- **indicator**: Add a _merge column to show row origin (left_only, right_only, both).
----------
**Tips to Remember**
- **merge** = Inner join ,Outer join,Left/Right join ,Many-to-many
- **suffixes** = to avoid same column name conflicts
- Indexes get dropped: merge ignores row indexes unless you use left_index or right_index (not shown here, but we can explore if you want!).

In [ ]:
df1 = pd.DataFrame({
    "key": ["b", "b", "a", "c", "a", "a", "b"],
    "data1": pd.Series(range(7), dtype="Int64")
})
df2 = pd.DataFrame({
    "key": ["a", "b", "d"],
    "data2": pd.Series(range(3), dtype="Int64")
})

pd.merge(df1 , df2) #By defaul inner join
pd.merge(df1 , df2 , on= 'key') #It is good to specify the key to avoid confusion
pd.merge(df1 , df2 , how = 'outer') #Here is how we define the join type

df3 = pd.DataFrame({
    "lkey": ["b", "b", "a", "c", "a", "a", "b"],
    "data1": pd.Series(range(7), dtype="Int64")
})
df4 = pd.DataFrame({
    "rkey": ["a", "b", "d"],
    "data2": pd.Series(range(3), dtype="Int64")
})

pd.merge(df3 , df4 , left_on = 'lkey' , right_on = 'rkey') #Using different keys
# You can use multiple keys too , you can use suffixes to avoid confusion between the same named columns

**Key Points to Remember**
- **Index as Merge Key**: When the merge key is in index then left_index = True and right_index = True is used
- **Hierarchical Index**: Pass multiple columns in the list left_on , right_on
- **join vs merge**: Join is shortcut for index based and index with column and the default is left join
- **Preserving Index**: In join the index of left dataframe is preserved whereas in merge the key/index of both dataframe may be dropped

In [ ]:
#Key and index based merging
left1 = pd.DataFrame({
    "key": ["a", "b", "a", "a", "b", "c"],
    "value": pd.Series(range(6), dtype="Int64")
})
right1 = pd.DataFrame({
    "group_val": [3.5, 7]
}, index=["a", "b"])
result = pd.merge(left1 , right1 , left_on = 'key' , right_index = True)

#When there is Heirchal index and multicolumns of key
lefth = pd.DataFrame({
"key1": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada"],
"key2": [2000, 2001, 2002, 2001, 2002],
"data": pd.Series(range(5), dtype="Int64")
})
righth_index = pd.MultiIndex.from_arrays([
    ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
    [2001, 2000, 2000, 2000, 2001, 2002]
])
righth = pd.DataFrame({
"event1": pd.Series([0, 2, 4, 6, 8, 10], dtype="Int64", index=righth_in
 dex),
"event2": pd.Series([1, 3, 5, 7, 9, 11], dtype="Int64", index=righth_in
 dex)
 })
result = pd.merge(lefth , righth , left_on = ['key1','key2'],right_index = True)

#index to index merging
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
index=["a", "c", "e"],
columns=["Ohio", "Nevada"]).astype("Int64")
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
 index=["b", "c", "d", "e"],
 columns=["Missouri", "Alabama"]).astype("Int64")
result = pd.merge(left2 , right2 , left_index = True , right_index = True)

#Now we are gonna see shortcuts using Join
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
 index=["a", "c", "e", "f"],
 columns=["New York", "Oregon"])
result = left2.join(right2 , how='outer')
result = left1.join(right1 , on='key')
resutl = left2.join([right2 , another] , how = 'outer') #Multiple dataframe

**The arguments of pd.concat**
- **objs**	List ya dictionary of pandas objects (Series/DataFrame) jo jodne hain. Yeh required hai.
- **axis**	Jodne ka axis: "index" (rows, default) ya "columns".
- **join**	"inner" (common indexes) ya "outer" (all indexes, default).
- **keys**	Hierarchical index banane ke liye values, jo batate hain kaunsa data kahan se aaya.
- **levels**	Specific indexes for hierarchical levels agar keys pass kiye hain.
- **names**	Hierarchical levels ke naam.
- **verify_integrity**	Agar True, to check karta hai ki new axis mein duplicates nahi hain. Default False.
- **ignore_index**	Agar True, to original indexes hata kar naye sequential index banata hai.
-------
For numpy we have np.concatenation

In [ ]:
''''Here are the examples of numpy arrays concatenation'''
data = np.arange(12).reshape((3,4))
result = np.concatenate([data,data],axis = 0) #Concatenating along axis 0

'''Here are the examples of pandas concat'''
s1 = pd.Series([0,1],index = ['a','b'],dtype = 'Int64')
s2 = pd.Series([2,3,4],index = ['c','d','e'],dtype = 'Int64')
s3 = pd.Series([5,6],index = ['f','g'],dtype = 'Int64')
result = pd.concat([s1,s2,s3]) #Concatenating along rows
result = pd.concat([s1,s2,s3],axis = 'columns') #Concatenating along columns
s4 = pd.concat([s1 , s3])
result = pd.concat([s1 , s4],join = 'inner',axis = 'columns') #To have only the matching indexes
result  = pd.concat([s1,s4],axis = 'columns')  #To have all the indexes

#Heirchal index concate
result  = pd.concat([s1,s1,s3],keys = ['one','two','three']) #default axis = 0 , index heirarchy
result.unstack() #Unstacking to convert to dataframe
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=["a", "b", "c"], columns=["one", "two"])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=["a", "c"], columns=["three", "four"])
result = pd.concat([df1 , df2],axis = 'columns',keys = ['level1','level2']) #Concatenating along columns with keys
result = pd.concat([df1 , df2],axis = 'columns',keys = ['level1','level2'],names = ['upper wala','neeche wala']) #Concatenating along columns and column heirarchy with names

#Dictionary method
result = pd.concat({'level1': df1, 'level2': df2}, axis='columns') #Concatenating along columns with dictionary method
#ignore index
result = pd.concat([df1 , df2],axis = 'columns',ignore_index = True) #Concatenating along columns and ignoring the index